In [94]:
print(classification_report(y_test, preds))
print(fps_adam)
print('mAP: ' + mAP_adam)
print ("Total number of parameters: " + str(params))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       300
           1       0.80      0.99      0.89       300
           2       1.00      0.75      0.86       300

   micro avg       0.92      0.92      0.92       900
   macro avg       0.93      0.92      0.91       900
weighted avg       0.93      0.92      0.91       900
 samples avg       0.92      0.92      0.92       900

FPS: 359.80 fps
mAP: 0.9437037037037036
Total number of parameters: 16499


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch import Tensor
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import matplotlib.image as mpimg
import pandas as pd
from numpy import prod
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelBinarizer
import time
import glob
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

import warnings
"""Optional[...] is a shorthand notation for Union[..., None], 
telling the type checker that either an object of the specific type is required, 
or None is required. ... stands for any valid type hint, 
including complex compound types or a Union[] of more types. 
Whenever you have a keyword argument with default value None, 
you should use Optional."""
from torch.jit.annotations import Optional, Tuple

from torch.hub import load_state_dict_from_url

from torchsummary import summary
from tqdm import tqdm

In [2]:
__all__ = ['GoogLeNet', 'googlenet', "GoogLeNetOutputs", "_GoogLeNetOutputs"]

model_urls = {
    # GoogLeNet ported from TensorFlow
    'googlenet': 'https://download.pytorch.org/models/googlenet-1378be20.pth',
}

GoogLeNetOutputs = namedtuple('GoogLeNetOutputs', ['logits', 'aux_logits2', 'aux_logits1'])
GoogLeNetOutputs.__annotations__ = {'logits': Tensor, 'aux_logits2': Optional[Tensor],
                                    'aux_logits1': Optional[Tensor]}

# Script annotations failed with _GoogleNetOutputs = namedtuple ...
# _GoogLeNetOutputs set here for backwards compat
_GoogLeNetOutputs = GoogLeNetOutputs


def googlenet(pretrained=False, progress=True, **kwargs):
    r"""GoogLeNet (Inception v1) model architecture from
    `"Going Deeper with Convolutions" <http://arxiv.org/abs/1409.4842>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        aux_logits (bool): If True, adds two auxiliary branches that can improve training.
            Default: *False* when pretrained is True otherwise *True*
        transform_input (bool): If True, preprocesses the input according to the method with which it
            was trained on ImageNet. Default: *False*
    """
    if pretrained:
        if 'transform_input' not in kwargs:
            kwargs['transform_input'] = True
        if 'aux_logits' not in kwargs:
            kwargs['aux_logits'] = False
        if kwargs['aux_logits']:
            warnings.warn('auxiliary heads in the pretrained googlenet model are NOT pretrained, '
                          'so make sure to train them')
        original_aux_logits = kwargs['aux_logits']
        kwargs['aux_logits'] = False
        kwargs['init_weights'] = True
        model = GoogLeNet(**kwargs)
        state_dict = load_state_dict_from_url(model_urls['googlenet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        if not original_aux_logits:
            model.aux_logits = False
            model.aux1 = None
            model.aux2 = None
        return model

    return Network(**kwargs)


class NetworkBN(nn.Module):
    __constants__ = ['aux_logits', 'transform_input']

    def __init__(self, num_classes=3, aux_logits=False, transform_input=False, init_weights=None,
                 blocks=None):
        super(NetworkBN, self).__init__()
        if blocks is None:
            blocks = [BasicConv2d, Inception, InceptionAux]
        if init_weights is None:
            warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '
                          'torchvision. If you wish to keep the old behavior (which leads to long initialization times'
                          ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
            init_weights = True
        assert len(blocks) == 3
        conv_block = blocks[0]
        inception_block = blocks[1]
        inception_aux_block = blocks[2]

        self.aux_logits = aux_logits
        self.transform_input = transform_input

        self.conv1 = conv_block(3, 64, kernel_size=7, stride=2, padding=3)
        self.lrn = nn.LocalResponseNorm(5, alpha = 9.9999997e-05)
        self.conv2 = conv_block(64, 64, kernel_size=1)
        self.lrn2 = nn.LocalResponseNorm(5, alpha = 9.9999997e-05)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = inception_block(64, 8, 16, 28)
        
        if aux_logits:
            self.aux1 = inception_aux_block(512, num_classes)
            self.aux2 = inception_aux_block(528, num_classes)
        else:
            self.aux1 = None
            self.aux2 = None

        self.avgpool = nn.AdaptiveAvgPool2d((4))
        self.dropout = nn.Dropout(0.40000001)
        self.fc = nn.Linear(384, num_classes)
        #self.softmax = nn.LogSoftmax()

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                X = stats.truncnorm(-2, 2, scale=0.01)
                values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
                values = values.view(m.weight.size())
                with torch.no_grad():
                    m.weight.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _transform_input(self, x):
        # type: (Tensor) -> Tensor
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0].clone(), 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1].clone(), 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2].clone(), 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
        return x

    def _forward(self, x):
        # type: (Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]
        # N x 3 x 224 x 224
        # already in conv block
        # x = F.relu(self.conv1(x))
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = F.max_pool2d(x, kernel_size = 3, stride = 2, ceil_mode=True) 
        x = self.lrn(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        aux1 = torch.jit.annotate(Optional[Tensor], None)
        if self.aux1 is not None:
            if self.training:
                aux1 = self.aux1(x)
        x = self.lrn2(x)
        # N x 64 x 56 x 56
        x = self.maxpool2(x)

        # N x 64 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        
        aux2 = torch.jit.annotate(Optional[Tensor], None)
        if self.aux2 is not None:
            if self.training:
                aux2 = self.aux2(x)
        
        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        #x = self.softmax(x)
        # N x 1000 (num_classes)
        #F.log_softmax(x)
        return x, aux2, aux1

    @torch.jit.unused
    def eager_outputs(self, x, aux2, aux1):
        # type: (Tensor, Optional[Tensor], Optional[Tensor]) -> GoogLeNetOutputs
        if self.training and self.aux_logits:
            return _GoogLeNetOutputs(x, aux2, aux1)
        else:
            return x

    def forward(self, x):
        # type: (Tensor) -> GoogLeNetOutputs
        x = self._transform_input(x)
        x, aux1, aux2 = self._forward(x)
        aux_defined = self.training and self.aux_logits
        if torch.jit.is_scripting():
            if not aux_defined:
                warnings.warn("Scripted GoogleNet always returns GoogleNetOutputs Tuple")
            return GoogLeNetOutputs(x, aux2, aux1)
        else:
            return self.eager_outputs(x, aux2, aux1)


class Inception(nn.Module):

    def __init__(self, in_channels, ch1x1, ch3x3red, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
            
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = conv_block(in_channels, ch3x3red, kernel_size=1)


    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)


        outputs = [branch1, branch2]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)


class InceptionAux(nn.Module):

    def __init__(self, in_channels, num_classes, conv_block=None):
        super(InceptionAux, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.conv = conv_block(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = F.adaptive_avg_pool2d(x, (4, 4))
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        # N x 1024
        x = F.dropout(x, 0.7, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x 1000 (num_classes)

        return x


class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x, inplace=False)
        return x

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
    print(f"Number of GPU's available : {torch.cuda.device_count()}")
    print(f"GPU device name : {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using CPU instead")
    device = torch.device("cpu")
    use_cuda = False

Number of GPU's available : 1
GPU device name : GeForce RTX 2080 Ti


In [4]:
model = NetworkBN().to(device)
summary(model, input_size=(3, 224, 224), batch_size=128, device = str(torch.device("cuda")))

<ipython-input-2-9bea0d24b132>:61: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
       BasicConv2d-3        [128, 64, 112, 112]               0
 LocalResponseNorm-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]           4,096
       BatchNorm2d-6          [128, 64, 56, 56]             128
       BasicConv2d-7          [128, 64, 56, 56]               0
 LocalResponseNorm-8          [128, 64, 56, 56]               0
         MaxPool2d-9          [128, 64, 28, 28]               0
           Conv2d-10           [128, 8, 28, 28]             512
      BatchNorm2d-11           [128, 8, 28, 28]              16
      BasicConv2d-12           [128, 8, 28, 28]               0
           Conv2d-13          [128, 16, 28, 28]           1,024
      BatchNorm2d-14          [128, 16,

In [5]:
model

NetworkBN(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lrn): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lrn2): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch2): BasicConv2d(
      (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps

In [6]:
__all__ = ['GoogLeNet', 'googlenet', "GoogLeNetOutputs", "_GoogLeNetOutputs"]

model_urls = {
    # GoogLeNet ported from TensorFlow
    'googlenet': 'https://download.pytorch.org/models/googlenet-1378be20.pth',
}

GoogLeNetOutputs = namedtuple('GoogLeNetOutputs', ['logits', 'aux_logits2', 'aux_logits1'])
GoogLeNetOutputs.__annotations__ = {'logits': Tensor, 'aux_logits2': Optional[Tensor],
                                    'aux_logits1': Optional[Tensor]}

# Script annotations failed with _GoogleNetOutputs = namedtuple ...
# _GoogLeNetOutputs set here for backwards compat
_GoogLeNetOutputs = GoogLeNetOutputs

def googlenet(pretrained=False, progress=True, **kwargs):
    r"""GoogLeNet (Inception v1) model architecture from
    `"Going Deeper with Convolutions" <http://arxiv.org/abs/1409.4842>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        aux_logits (bool): If True, adds two auxiliary branches that can improve training.
            Default: *False* when pretrained is True otherwise *True*
        transform_input (bool): If True, preprocesses the input according to the method with which it
            was trained on ImageNet. Default: *False*
    """
    if pretrained:
        if 'transform_input' not in kwargs:
            kwargs['transform_input'] = True
        if 'aux_logits' not in kwargs:
            kwargs['aux_logits'] = False
        if kwargs['aux_logits']:
            warnings.warn('auxiliary heads in the pretrained googlenet model are NOT pretrained, '
                          'so make sure to train them')
        original_aux_logits = kwargs['aux_logits']
        kwargs['aux_logits'] = False
        kwargs['init_weights'] = False
        model = GoogLeNet(**kwargs)
        state_dict = load_state_dict_from_url(model_urls['googlenet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        if not original_aux_logits:
            model.aux_logits = False
            model.aux1 = None
            model.aux2 = None
        return model

    return Network(**kwargs)


class Network(nn.Module):
    __constants__ = ['aux_logits', 'transform_input']

    def __init__(self, num_classes=3, aux_logits=False, transform_input=False, init_weights=None,
                 blocks=None):
        super(Network, self).__init__()
        if blocks is None:
            blocks = [BasicConv2d, Inception, InceptionAux]
        if init_weights is None:
            warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '
                          'torchvision. If you wish to keep the old behavior (which leads to long initialization times'
                          ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
            init_weights = True
        assert len(blocks) == 3
        conv_block = blocks[0]
        inception_block = blocks[1]
        inception_aux_block = blocks[2]

        self.aux_logits = aux_logits
        self.transform_input = transform_input

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.lrn = nn.LocalResponseNorm(5, alpha = 9.9999997e-05)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.lrn2 = nn.LocalResponseNorm(5, alpha = 9.9999997e-05)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = inception_block(64, 8, 16, 28)
        
        if aux_logits:
            self.aux1 = inception_aux_block(512, num_classes)
            self.aux2 = inception_aux_block(528, num_classes)
        else:
            self.aux1 = None
            self.aux2 = None

        self.avgpool = nn.AdaptiveAvgPool2d((4))
        self.dropout = nn.Dropout(0.40000001)
        #2048
        self.fc = nn.Linear(384, num_classes)
        #self.softmax = nn.LogSoftmax()

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                X = stats.truncnorm(-2, 2, scale=0.01)
                values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
                values = values.view(m.weight.size())
                with torch.no_grad():
                    m.weight.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _transform_input(self, x):
        # type: (Tensor) -> Tensor
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0].clone(), 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1].clone(), 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2].clone(), 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
        return x

    def _forward(self, x):
        # type: (Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]
        # N x 3 x 224 x 224
        # already in conv block
        x = F.relu(self.conv1(x), inplace = True)
        #x = self.conv1(x)
        # N x 64 x 112 x 112
        x = F.max_pool2d(x, kernel_size = 3, stride = 2, ceil_mode=True) 
        x = self.lrn(x)
        # N x 64 x 56 x 56
        x = F.relu(self.conv2(x), inplace = True)
        aux1 = torch.jit.annotate(Optional[Tensor], None)
        if self.aux1 is not None:
            if self.training:
                aux1 = self.aux1(x)
        x = self.lrn2(x)
        # N x 64 x 56 x 56
        x = self.maxpool2(x)

        # N x 64 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        
        aux2 = torch.jit.annotate(Optional[Tensor], None)
        if self.aux2 is not None:
            if self.training:
                aux2 = self.aux2(x)
        # N x 256 x 28 x 28
        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        #x = self.softmax(x)
        # N x 1000 (num_classes)
        return x, aux2, aux1

    @torch.jit.unused
    def eager_outputs(self, x, aux2, aux1):
        # type: (Tensor, Optional[Tensor], Optional[Tensor]) -> GoogLeNetOutputs
        if self.training and self.aux_logits:
            return _GoogLeNetOutputs(x, aux2, aux1)
        else:
            return x

    def forward(self, x):
        # type: (Tensor) -> GoogLeNetOutputs
        x = self._transform_input(x)
        x, aux1, aux2 = self._forward(x)
        aux_defined = self.training and self.aux_logits
        if torch.jit.is_scripting():
            if not aux_defined:
                warnings.warn("Scripted GoogleNet always returns GoogleNetOutputs Tuple")
            return GoogLeNetOutputs(x, aux2, aux1)
        else:
            return self.eager_outputs(x, aux2, aux1)

class Inception(nn.Module):

   # def __init__(self, in_channels, ch1x1, ch3x3red, pool_proj):
   #    super(Inception, self).__init__()
        
   #    self.branch1 = nn.ReLU(nn.Conv2d(in_channels, ch1x1, kernel_size=1))

   #    self.branch2 = nn.ReLU(nn.Conv2d(in_channels, ch3x3red, kernel_size=1))

    def __init__(self, in_channels, ch1x1, ch3x3red, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
            
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = conv_block(in_channels, ch3x3red, kernel_size=1)

    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)


        outputs = [branch1, branch2]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

class InceptionAux(nn.Module):

    def __init__(self, in_channels, num_classes, conv_block=None):
        super(InceptionAux, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.conv = conv_block(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = F.adaptive_avg_pool2d(x, (4, 4))
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        # N x 1024
        x = F.dropout(x, 0.7, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x 1000 (num_classes)

        return x


class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x, inplace=False)
        return x

In [7]:
torch.manual_seed(42)
network = Network()
network

<ipython-input-6-0bc76ad473e5>:60: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


Network(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (lrn): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (lrn2): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch2): BasicConv2d(
      (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (avgpool): AdaptiveAvgPool2d(output_size=4)
  (dropout): Dropout(p=0.40000001, inplace=False)
  (fc): Linear(in_features=384, out_features=3, bias=True)
)

In [8]:
model = Network().to(device)
summary(model, input_size=(3, 224, 224), batch_size=128, device = str(torch.device("cuda")))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,472
 LocalResponseNorm-2          [128, 64, 56, 56]               0
            Conv2d-3          [128, 64, 56, 56]           4,160
 LocalResponseNorm-4          [128, 64, 56, 56]               0
         MaxPool2d-5          [128, 64, 28, 28]               0
            Conv2d-6           [128, 8, 28, 28]             512
       BatchNorm2d-7           [128, 8, 28, 28]              16
       BasicConv2d-8           [128, 8, 28, 28]               0
            Conv2d-9          [128, 16, 28, 28]           1,024
      BatchNorm2d-10          [128, 16, 28, 28]              32
      BasicConv2d-11          [128, 16, 28, 28]               0
        Inception-12          [128, 24, 28, 28]               0
AdaptiveAvgPool2d-13            [128, 24, 4, 4]               0
          Dropout-14                 [1

<ipython-input-6-0bc76ad473e5>:60: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


In [9]:
def get_num_correct(preds, labels):
      return preds.argmax(dim=1).eq(labels).sum().item()

In [10]:
# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple("Run", params.keys())
        
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

In [11]:
# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json!@!
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        # tracking every run count, run data, hyper-params used, time
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
    
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment = f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        #self.tb.add_image('images', grid)
        #self.tb.add_graph(self.network, images)
        
    def end_run(self, model):
        
        MODEL_PATH = "./weights/model_{}.pt".format(self)
        torch.save(model, MODEL_PATH)
        
        self.tb.close()
        self.epoch_count = 0
       
       # zero epoch count, loss, accuracy, 
    def begin_epoch(self):
        self.epoch_start_time = time.time()

        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    # 
    def end_epoch(self, model):
        # calculate epoch duration and run duration(accumulate)
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        # record epoch loss and accuracy
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        # Record epoch loss and accuracy to TensorBoard 
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        # Record params to TensorBoard
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    

        # Write into 'results' (OrderedDict) for all run related data
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        MODEL_PATH = "./weights/model_{}.pt".format(self.epoch_count)
        torch.save(model, MODEL_PATH)
        
        # Record hyper-params into 'results'
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

        # display epoch information and show progress
        clear_output(wait=True)
        display(df)

      # accumulate loss of batch into entire epoch loss
    def track_loss(self, loss):
        # multiply batch size so variety of batch sizes can be compared
        self.epoch_loss += loss.item() * self.loader.batch_size

      # accumulate number of corrects of batch into entire epoch num_correct
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    @torch.no_grad()
    def _get_num_correct(self, preds, lables):
        return preds.argmax(dim = 1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
        self.run_data,
        orient = "columns").to_csv(f'{fileName}.csv')
        
        
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [12]:
train_set = torchvision.datasets.ImageFolder(
    root = './Data_cleaned/train'
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)


In [13]:
%%time
from tqdm import tqdm_notebook

loader = torch.utils.data.DataLoader(
    train_set, batch_size = 512, num_workers = 2, pin_memory = True
)
data = next(iter(loader))
mean = data[0].mean()
std = data[0].std()
mean, std

#nb_samples = 0.
#channel_mean = torch.Tensor([0., 0., 0.])
#channel_std = torch.Tensor([0., 0., 0.])
#for images in tqdm_notebook(loader):
    # scale image to be between 0 and 1 
    
#    images = np.asarray(images)/np.asarray(255.)
#    batch_samples = images.size(0)
#    images = images.view(batch_samples, images.size(1)*images.size(2), 3)
#    for i in range(3):
#        channel_mean[i]+=images[:, :,i].mean(1).sum(0)
#        channel_std[i]+=images[:, :,i].std(1).sum(0)
#    nb_samples += batch_samples

#channel_mean /= nb_samples
#channel_std /= nb_samples


CPU times: user 263 ms, sys: 310 ms, total: 573 ms
Wall time: 2.09 s


(tensor(0.4738), tensor(0.2598))

In [14]:
train_set_normal = torchvision.datasets.ImageFolder(
    root = './Data_cleaned/train'
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)


In [15]:
trainsets = {
    'not_normal': train_set,
    'normal' : train_set_normal
}

In [16]:
networkBN = NetworkBN()
#network2 = Network()

<ipython-input-2-9bea0d24b132>:61: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


In [17]:
networks = {
#    'no_batch_norm': network,
    'batch_norm': networkBN
}

In [18]:
CURRENT_DIR = '/home/vadim/testovoe/dev/'

In [19]:
test_df = pd.read_csv(CURRENT_DIR+"Data/text.txt", sep="\s+", header=None, names=["name", "category"])

In [20]:
def get_pad_width(im, new_shape, is_rgb=True):
    pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    
    return im

In [21]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['name'])):
    x_test[i, :, :, :] = preprocess_image(
         f'/home/vadim/testovoe/dev/{image_id}'
    )

100%|██████████| 900/900 [00:00<00:00, 1071.24it/s]


In [22]:
y_test = pd.get_dummies(test_df['category']).values

In [23]:
%%time
# Test different configurations
# for every run [value] that is going to be used e.g [.001, .01] = two runs
# the bigger the batch the less gradient updates steps made
params = OrderedDict(
    lr = [.0003],
    # max batch size 352
    batch_size = [128],
    weight_decay = [3.0000001e-06],
    num_workers = [1],
    device = ["cuda"],
    trainset = ["normal"],
    # try all the values in the dict network1, network2
    network = list(networks.keys())
)
m = RunManager()
# active run or current run
# this thing detecting RuntimeError: Function 'LogSoftmaxBackward' returned nan values in its 0th output. 
# if k = 0 in LRN layers
torch.autograd.set_detect_anomaly(True)
for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    # redefine the network
    network = networks[run.network].to(device)
    network.train()
    loader = DataLoader(trainsets[run.trainset], batch_size = run.batch_size, num_workers = run.num_workers, 
                       pin_memory = True) 
    optimizer = optim.Adam(network.parameters(), lr = run.lr, weight_decay = run.weight_decay) 
    
    m.begin_run(run, network, loader)
    for epoch in range(100):
        m.begin_epoch()
        for batch in loader:
            #network.train()
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            #7.9
            #optimizer.zero_grad()
            #8 sec
            for p in network.parameters(): p.grad = None
            loss.backward() # Calculate gradients
            optimizer.step() # Update Weights
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
        m.end_epoch(network)
    m.end_run(network)
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,weight_decay,num_workers,device,trainset,network
0,1,1,0.749283,0.762316,55.870668,56.533402,0.0003,128,0.000003,1,cuda,normal,batch_norm
1,1,2,0.874722,0.681649,57.136831,113.748542,0.0003,128,0.000003,1,cuda,normal,batch_norm
2,1,3,0.872166,0.679366,57.244528,171.064068,0.0003,128,0.000003,1,cuda,normal,batch_norm
3,1,4,0.849633,0.677885,56.928435,228.064021,0.0003,128,0.000003,1,cuda,normal,batch_norm
4,1,5,0.797469,0.694790,57.801470,285.946223,0.0003,128,0.000003,1,cuda,normal,batch_norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,96,0.019135,0.995928,68.799151,5907.251275,0.0003,128,0.000003,1,cuda,normal,batch_norm
96,1,97,0.018024,0.996082,68.771111,5976.116111,0.0003,128,0.000003,1,cuda,normal,batch_norm
97,1,98,0.017682,0.995681,69.345453,6045.554350,0.0003,128,0.000003,1,cuda,normal,batch_norm
98,1,99,0.016845,0.996021,64.861036,6110.502424,0.0003,128,0.000003,1,cuda,normal,batch_norm


CPU times: user 44min 23s, sys: 5min 47s, total: 50min 10s
Wall time: 1h 42min 56s


In [24]:
MODEL_PATH = "./weights/model_99.pt"
#model = NetworkBN()
model = torch.load(MODEL_PATH)
#model.eval()

In [25]:
pd.DataFrame.from_dict(m.run_data).sort_values("accuracy", ascending = False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,weight_decay,num_workers,device,trainset,network
99,1,100,0.014834,0.997039,66.258320,6176.852132,0.0003,128,0.000003,1,cuda,normal,batch_norm
96,1,97,0.018024,0.996082,68.771111,5976.116111,0.0003,128,0.000003,1,cuda,normal,batch_norm
98,1,99,0.016845,0.996021,64.861036,6110.502424,0.0003,128,0.000003,1,cuda,normal,batch_norm
95,1,96,0.019135,0.995928,68.799151,5907.251275,0.0003,128,0.000003,1,cuda,normal,batch_norm
94,1,95,0.019787,0.995774,69.460165,5838.355963,0.0003,128,0.000003,1,cuda,normal,batch_norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1,6,0.737526,0.724774,58.075132,344.097553,0.0003,128,0.000003,1,cuda,normal,batch_norm
4,1,5,0.797469,0.694790,57.801470,285.946223,0.0003,128,0.000003,1,cuda,normal,batch_norm
1,1,2,0.874722,0.681649,57.136831,113.748542,0.0003,128,0.000003,1,cuda,normal,batch_norm
2,1,3,0.872166,0.679366,57.244528,171.064068,0.0003,128,0.000003,1,cuda,normal,batch_norm


In [26]:
categories=[x.strip() for x in open('labels_adam.txt').readlines()]

In [79]:
%%time
# This could be done by defining separate dataloader for test dataset
preds_adam = []
preds = []
from timeit import default_timer as timer
time_start = timer()
device = torch.device("cuda")
# this is untrained
# uncomment for 67% accu
#model = networks['batch_norm'].to(device)
#model = Network().to(device)
#model = model.to(device)
def get_preds(network):
        network = network
        # this is very ineffective O(n^2)+complexity of network
        # model.eval() will notify all your layers that you are in eval mode, that way, 
        # batchnorm or dropout layers will work in eval mode instead of training mode.
        # torch.no_grad() impacts the autograd engine and deactivate it. 
        # It will reduce memory usage and speed up computations but you won’t be able to backprop 
        # (which you don’t want in an eval script).

        with torch.no_grad():
            for i, x in enumerate(categories):
                for j, y in enumerate(sorted(glob.glob('Data/test/{}/*'.format(x)))):
                    im = Image.open(y)
                    image = transforms.ToTensor()(im).unsqueeze_(0).to(device)
                    output = model(image) # preds
                    preds = output.argmax(dim=1, keepdim=True)
                    preds_adam.extend(preds.cpu().numpy())
                    #preds_adam.append(preds.detach().cpu().clone().numpy())
                    #print(preds_adam)
                    #preds_np = pred.detach().cpu().clone().numpy()
                    #print(preds_np)
                    #preds_np.append(pred.detach().cpu().clone().numpy())
                    #preds_adam.extend(pred.cpu().numpy())
                    #print(preds)
                    #print(preds.argmax(dim = 1))
                    #preds_np.append(preds.detach().cpu().clone().numpy())
                    #preds_adam.append(np.argmax(preds_np))
                    #print(y, categories[np.argmax(pred['softmax'])])
        return preds_adam
preds_adam = get_preds(network)
time_end = timer()
fps_adam=('FPS: %.2f fps' % (1000/(time_end-time_start)))
print('FPS: %.2f fps' % (1000/(time_end-time_start)))

FPS: 359.80 fps
CPU times: user 2.76 s, sys: 20.7 ms, total: 2.78 s
Wall time: 2.78 s


In [28]:
%%time
# This could be done by defining separate dataloader for test dataset
preds_adam = []
preds = []
from timeit import default_timer as timer
time_start = timer()
device = torch.device("cuda")
def predictions():
    for i, x in enumerate(categories):
        for j, y in enumerate(sorted(glob.glob('Data/test/{}/*'.format(x)))):
            im = Image.open(y)
            image = transforms.ToTensor()(im).unsqueeze_(0).to(device)
            output = model(image) # preds
            #print(output)
            preds = output.argmax(dim=1, keepdim=True)
            #print(categories[np.argmax(output)])
            preds_adam.extend(preds.cpu().numpy())
            #preds_adam.append(preds.detach().cpu().clone().numpy())
            #print(preds_adam)
            #preds_np = pred.detach().cpu().clone().numpy()
            #print(preds_np)
            #preds_np.append(pred.detach().cpu().clone().numpy())
            #preds_adam.extend(pred.cpu().numpy())
            #print(preds)
            #print(preds.argmax(dim = 1))
            #preds_np.append(preds.detach().cpu().clone().numpy())
            #preds_adam.append(np.argmax(preds_np))
            #print(y, categories[np.argmax(pred['softmax'])])
    return preds_adam
preds_adam = predictions()
time_end = timer()
fps=('FPS: %.2f fps' % (1000/(time_end-time_start)))
print('FPS: %.2f fps' % (1000/(time_end-time_start)))

FPS: 61.68 fps
CPU times: user 15 s, sys: 1.07 s, total: 16 s
Wall time: 16.2 s


In [29]:
# list of numpy arrays
preds_adam

[array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),

In [30]:
def extractDigits(lst): 
    return list(map(lambda el:el.tolist(), lst)) 

In [31]:
extractDigits(preds_adam)
preds_adam

[array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),

In [32]:
flat_list = [item for sublist in preds_adam for item in sublist]

In [33]:
flat_list

[0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,


In [34]:
lb = LabelBinarizer()
preds_adam = lb.fit_transform(flat_list)
preds_adam

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [35]:
print(classification_report(y_test, preds_adam))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       300
           1       0.64      0.42      0.51       300
           2       0.55      0.81      0.66       300

   micro avg       0.64      0.64      0.64       900
   macro avg       0.65      0.64      0.63       900
weighted avg       0.65      0.64      0.63       900
 samples avg       0.64      0.64      0.64       900



In [36]:
mAP_adam = str(average_precision_score(y_test, preds_adam, average="samples"))
print(average_precision_score(y_test, preds_adam, average="samples"))

0.7570370370370371


In [81]:
test_set = torchvision.datasets.ImageFolder(
    root = './Data/test'
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)


In [82]:
test_loader = DataLoader(test_set, batch_size = 352, num_workers = 1, 
                       pin_memory = True) 

In [83]:
def test(model, optimizer, test_loader):
    """
    Args:
        model : network
        test_loader : pytorch test loader
    Returns:
        list : predicted values
    """
    
    model = torch.load(MODEL_PATH)
    optimizer = optimizer
    preds = []

    with torch.no_grad():
        for batch in test_loader:
            images = batch[0].to(device)
            labels = batch[1].to(device)
            output = model(images)
            pred = output.argmax(dim=1, keepdim=True)
            preds.extend(pred.cpu().numpy())

    return preds

In [84]:
model = NetworkBN().to(device)
optimizer = optim.Adam(network.parameters(), lr = 0.0003)

# get predictions
predictions = test(torch.load(MODEL_PATH), optimizer, test_loader)

<ipython-input-2-9bea0d24b132>:61: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


In [85]:
preds = predictions

In [86]:
def extractDigits(lst): 
    return list(map(lambda el:el.tolist(), lst)) 
extractDigits(preds)
flat_list = []
flat_list = [item for sublist in preds for item in sublist]
flat_list

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [87]:
lb = LabelBinarizer()
preds = lb.fit_transform(flat_list)
preds

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [88]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       300
           1       0.80      0.99      0.89       300
           2       1.00      0.75      0.86       300

   micro avg       0.92      0.92      0.92       900
   macro avg       0.93      0.92      0.91       900
weighted avg       0.93      0.92      0.91       900
 samples avg       0.92      0.92      0.92       900



In [89]:
mAP_adam = str(average_precision_score(y_test, preds, average="samples"))
print(average_precision_score(y_test, preds, average="samples"))

0.9437037037037036


In [91]:
params = sum([param.nelement() for param in model.parameters()])